In [ ]:
import os
import json
import pandas as pd
from zipfile import ZipFile
from tensorflow import keras

In [ ]:
kaggle_dictionary = json.load(open("kaggle.json"))

In [ ]:
kaggle_dictionary.values

<function dict.values>

In [ ]:
kaggle_dictionary.keys()

dict_keys(['username', 'key'])

In [ ]:
# creating kaggle credentials as environmnet variables

os.environ["KAGGLE_USERNAME"]= kaggle_dictionary['username']
os.environ["KAGGLE_KEY"]= kaggle_dictionary["key"]

In [ ]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
imdb-dataset-of-50k-movie-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!ls

'IMDB Dataset.csv'   imdb-dataset-of-50k-movie-reviews.zip   kaggle.json   sample_data


In [ ]:
with ZipFile('/content/imdb-dataset-of-50k-movie-reviews.zip' , 'r') as zip_file:
 zip_file.extractall()

In [ ]:
!ls

'IMDB Dataset.csv'   imdb-dataset-of-50k-movie-reviews.zip   kaggle.json   sample_data


In [ ]:
data = pd.read_csv('/content/IMDB Dataset.csv')

In [ ]:
data.shape

(50000, 2)

In [ ]:
data[0:5]

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
data['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [ ]:
data.replace({"sentiment": { "positive": 1 , "negative": 0 }} , inplace=True)


In [ ]:
data['sentiment'].value_counts()

sentiment
1    25000
0    25000
Name: count, dtype: int64

In [ ]:
# spliting into train & test data
from sklearn.model_selection import train_test_split
train_data ,test_data = train_test_split(data , test_size =0.2 , random_state=42)

In [ ]:
train_data.head()

,review,sentiment
39087,That's what I kept asking myself during the ma...,0
30893,I did not watch the entire movie. I could not ...,0
45278,A touching love story reminiscent of In the M...,1
16398,This latter-day Fulci schlocker is a totally a...,0
13653,"First of all, I firmly believe that Norwegian ...",0


In [ ]:
# Preprocessing
# Tokenize the text data

from tensorflow import keras
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])
x_train = pad_sequences(tokenizer.texts_to_sequences(train_data['review']) , maxlen = 200)
x_test = pad_sequences(tokenizer.texts_to_sequences(test_data['review'])  , maxlen = 200 )

In [ ]:
print(x_train[-5:])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0 2825    1 1800    2    4    1   85
    17 1250    9   79  274    1  660    4   19    8   94 1245    5  753
     1  179    3  119 2603 3300 2759    2    5 3214   15  245    5  213
   420  249   36  369 1370    1  503    4    1 1310  443   80  354   11
    19   44    3   78   85  776   31  567  166 2234   36   80 22

In [ ]:
print(x_test)

[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [ ]:
y_train = train_data['sentiment']
y_test = test_data['sentiment']

In [ ]:
print(y_train)
print(y_test)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64
33553    1
9427     1
199      0
12447    1
39489    0
        ..
28567    0
25079    1
18707    1
15200    0
5857     1
Name: sentiment, Length: 10000, dtype: int64


LSTM Model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense , Embedding , LSTM

model = Sequential()
model.add(Embedding(input_dim = 5000 , output_dim = 128 , input_length = 200))
model.add(LSTM(128, dropout=0.1 , recurrent_dropout = 0.2))
model.add(Dense( 1 , activation  ='sigmoid'))

In [ ]:
model.summary( )

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 128)          640000    
                                                                 
 lstm (LSTM)                 (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 771713 (2.94 MB)
Trainable params: 771713 (2.94 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# COMPILE THE MODEL
model.compile(optimizer = 'adam' , loss = 'binary_crossentropy' , metrics = 'acc')

In [ ]:
# Training the Model
model.fit(x_train , y_train , epochs = 5 , batch_size = 64 , validation_split = 0.2)

Epoch 1/5
500/500 [==============================] - 352s 697ms/step - loss: 0.4069 - acc: 0.8148 - val_loss: 0.3256 - val_acc: 0.8608
Epoch 2/5
500/500 [==============================] - 346s 691ms/step - loss: 0.2725 - acc: 0.8896 - val_loss: 0.3183 - val_acc: 0.8683
Epoch 3/5
500/500 [==============================] - 344s 687ms/step - loss: 0.2198 - acc: 0.9130 - val_loss: 0.3003 - val_acc: 0.8741
Epoch 4/5
500/500 [==============================] - 362s 724ms/step - loss: 0.1869 - acc: 0.9283 - val_loss: 0.3797 - val_acc: 0.8730
Epoch 5/5
500/500 [==============================] - 378s 756ms/step - loss: 0.1615 - acc: 0.9390 - val_loss: 0.4363 - val_acc: 0.8575


In [ ]:
loss,accuracy = model.evaluate(x_test , y_test)
print('Loss' ,loss)
print('accuracy' , accuracy)

313/313 [==============================] - 26s 81ms/step - loss: 0.4067 - acc: 0.8718
Loss 0.40674591064453125
accuracy 0.8718000054359436


In [ ]:
def output_sentiment(review):

# tokenize and Pad the review

 sequence = tokenizer.texts_to_sequences([review])
 padded_sequences = pad_sequences(sequence , maxlen =200)
 prediction = model.predict(padded_sequences)
 review ='positive' if prediction > 0.5 else 'negative'
 return review

In [ ]:
#example
new_review = "movie was amazing and lovely "
review = output_sentiment(new_review)
print(review)

1/1 [==============================] - 0s 48ms/step
positive
